In [115]:
; cl-waffeを読み込む
(load "../../cl-waffe.asd")
(ql:quickload :cl-waffe :silent t)
(use-package :cl-waffe)
(use-package :cl-waffe.nn)

T

(:CL-WAFFE)

T

T

SB-KERNEL:REDEFINITION-WITH-DEFMETHOD: redefining PERFORM (#<STANDARD-CLASS ASDF/LISP-ACTION:TEST-OP>
                                                           #<SB-MOP:EQL-SPECIALIZER #<SYSTEM "cl-waffe/test">>) in DEFMETHOD


In [116]:
(defmodel Encoder (vocab-size embedding-dim hidden-size)
  :parameters ((embedding (Embedding vocab-size embedding-dim :pad-idx 0))
               (layer     (RNN embedding-dim hidden-size :num-layers 1)))
  :forward ((x)
            (with-calling-layers x
                (embedding x)
                (layer x))))

(defmodel Decoder (vocab-size embedding-dim hidden-size)
  :parameters ((embedding (Embedding vocab-size embedding-dim :pad-idx 0))
               (layer     (RNN embedding-dim hidden-size :num-layers 1))
               (h2l       (linearlayer hidden-size vocab-size)))
  
  :forward ((encoder-state y)
            (let* ((ye (call (self embedding) y))
                   (hs (call (self layer) ye encoder-state))
                   (h-output (call (self h2l) hs)))
                  (list h-output hs))))

(defmodel Seq2Seq (vocab-size embedding-dim input-size)
  :parameters ((encoder (Encoder vocab-size embedding-dim input-size))
               (decoder (Decoder vocab-size embedding-dim input-size)))  
  :forward ((x y)
            (let ((x-state (call (self encoder) x))
                  (y1 (!zeros (!shape y))))
                 (setq y1 (setf (!aref y1 t) (!aref y '(1 t))))
                 (call (self decoder) x-state y1))))

NIL

NIL

NIL

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::COPY-ENCODER in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::ENCODER-P in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF COMMON-LISP-USER::ENCODER-MODEL-IDENT) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::ENCODER-MODEL-IDENT in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF COMMON-LISP-USER::ENCODER-HIDE-FROM-TREE) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::ENCODER-HIDE-FROM-TREE in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF COMMON-LISP-USER::ENCODER-FORWARD) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::ENCODER-FORWARD in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF COMMON-LISP-USER::ENCODER-OBJECT-TYPE) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::ENCODER-OBJECT-TYPE in DEFUN
SB-KERNEL:RED

In [117]:
(deftrainer Seq2SeqTrainer (vocab-size embedding-dim hidden-size)
  :model (Seq2Seq vocab-size embedding-dim hidden-size)
  :optimizer cl-waffe.optimizers:Adam
  :optimizer (:lr 1e-4)
  :step-model ((x y)
               (zero-grad)
               (let* ((outs (call (self model) x y))
                      (out (softmax-cross-entropy (car outs) y)))
                     (backward out)
                     (update)
                     out))
  :predict ((x) (call (model) x)))

NIL

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::COPY-SEQ2SEQTRAINER in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::SEQ2SEQTRAINER-P in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF COMMON-LISP-USER::SEQ2SEQTRAINER-MODEL) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::SEQ2SEQTRAINER-MODEL in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF COMMON-LISP-USER::SEQ2SEQTRAINER-OPTIMIZER) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::SEQ2SEQTRAINER-OPTIMIZER in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF COMMON-LISP-USER::SEQ2SEQTRAINER-OPTIMIZER-REPORT) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMON-LISP-USER::SEQ2SEQTRAINER-OPTIMIZER-REPORT in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining (COMMON-LISP:SETF COMMON-LISP-USER::SEQ2SEQTRAINER-PREDICT) in DEFUN
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining COMMO

In [118]:
(setq model (seq2seqtrainer 10 10 10))

<Dataset: SEQ2SEQTRAINER()>

SIMPLE-WARNING: undefined variable: CL-WAFFE:MODEL


In [122]:
(setq x (!ones `(2 3)))
(setq y (!ones `(2 3)))

(time (step-model model x y))

#Const(((1.0 1.0 1.0)
        (1.0 1.0 1.0)) :mgl t :shape (2 3) :backward NIL)

#Const(((1.0 1.0 1.0)
        (1.0 1.0 1.0)) :mgl t :shape (2 3) :backward NIL)

#Const(2.2968965)

SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::X
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::Y
SIMPLE-WARNING: undefined variable: CL-WAFFE:MODEL
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::X
SIMPLE-WARNING: undefined variable: COMMON-LISP-USER::Y

(#Const((((9.261... -0.01... ~ 0.023... 0.004...))
        ((9.261... -0.01... ~ 0.023... 0.004...))) :mgl t :shape (2 1 10) :backward <Node: AREFTENSOR{W309102}>)
 #Const((((9.261... -0.01... ~ 0.023... 0.004...))
        ((9.261... -0.01... ~ 0.023... 0.004...))) :mgl t :shape (2 1 10) :backward <Node: AREFTENSOR{W309103}>)
 #Const((((9.261... -0.01... ~ 0.023... 0.004...))
        ((9.261... -0.01... ~ 0.023... 0.004...))) :mgl t :shape (2 1 10) :backward <Node: AREFTENSOR{W309104}>)) 
#Const((((9.261... -0.01... ~ 0.023... 0.004...)         
                   ...
         (9.261... -0.01... ~ 0.023... 0.004...))
        ((9.261... -0.01... ~ 0.023... 0.004...)         
                   ...
         (9.261...